In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from sklearn.linear_model.logistic import LogisticRegression

data_path = os.path.abspath(os.path.dirname("a"))

vitals_train = pd.read_csv("%s/Training_Dataset/id_time_vitals_train.csv" % data_path)
labs_train = pd.read_csv("%s/Training_Dataset/id_time_labs_train.csv" % data_path)
age_train = pd.read_csv("%s/Training_Dataset/id_age_train.csv" % data_path)
label_train = pd.read_csv("%s/Training_Dataset/id_label_train.csv" % data_path)


In [ ]:
index_zero = labs_train[labs_train.TIME == 0].index.tolist()
for i in range(len(index_zero)):
  index_zero[i] -= 1
#print index_zero

'''
Other Way to determine "index_zero"
index_zero = numpy.array(index_zero)
index_zero = index_zero - 1
'''

age_train['TOTAL_TIME'] = np.nan
age_train.TOTAL_TIME[0:-1] = labs_train.TIME[index_zero[1:]]
age_train.TOTAL_TIME[-1:] = labs_train.TIME[-1:]
# age_train.head(10)
#print age_label

In [ ]:
# age_train2 = age_train.drop(['ID', 'AGE'], axis=1)
# age_train2_mean = np.array(age_train2.mean(axis=0))
# age_train2_std = np.array(age_train2.std(axis=0))
# thresTotalTime1 = (age_train2_mean - 1*age_train2_std)
# thresTotalTime2 = (age_train2_mean + 1*age_train2_std)
# thresTotalTime1_labels = pd.DataFrame(thresTotalTime1,age_train2.columns)
# thresTotalTime2_labels = pd.DataFrame(thresTotalTime2,age_train2.columns)

In [ ]:
# print age_train2_mean
# print age_train2_std
# plt.figure();
# age_train.TOTAL_TIME.plot(kind='hist', stacked=True, alpha=0.5, bins=100)

In [ ]:
# for col in age_train2.columns:
# #     print col
#     for row in range(age_train2.index.size):
# #         if age_train2[col][row] > thresTotalTime1_labels[0][col] and age_train2[col][row] < thresTotalTime2_labels[0][col]:
#         if age_train2[col][row] > thresTotalTime2_labels[0][col]:
#             age_train2[col][row] = 1
#         else:
#             age_train2[col][row] = 0
            
# age_train_normTime = pd.merge(age_train,age_train2, left_index=True,right_index=True, how='outer')

In [ ]:
# print age_train_normTime.head(100)
# print thresTotalTime1
# print thresTotalTime2

In [ ]:
# Merging vitals_train & labs_train
vitals_labs = pd.merge(vitals_train,labs_train, left_index=True,right_index=True, how='outer')
# vitals_labs.head()

In [ ]:
vitals_labs_icu = vitals_labs[vitals_labs['ICU'] == 1]
# vitals_labs_icu.head(20)

In [ ]:
vitals_labs2 = vitals_labs_icu.drop(['ID_x', 'TIME_x', 'ICU', 'ID_y', 'TIME_y'], axis=1)
vitals_labs2_mean = np.array(vitals_labs2.mean(axis=0))
vitals_labs2_std = np.array(vitals_labs2.std(axis=0))
# print vitals_labs2_mean
# print vitals_labs2_std

In [ ]:
thres1 = (vitals_labs2_mean - 2*vitals_labs2_std)
thres2 = (vitals_labs2_mean + 2*vitals_labs2_std)
thres1_labels = pd.DataFrame(thres1,vitals_labs2.columns)
thres2_labels = pd.DataFrame(thres2,vitals_labs2.columns)
# thres1
# thres_labels

In [ ]:
# for col in vitals_labs2.columns:
#     plt.figure();
#     vitals_labs2.col.plot(kind='hist', stacked=True, alpha=0.5, bins=100)

In [ ]:
for col in vitals_labs2.columns:
    print col
    for row in vitals_labs2.index:
        if vitals_labs2[col][row] > thres1_labels[0][col] and vitals_labs2[col][row] < thres2_labels[0][col]:
            vitals_labs2[col][row] = 0
        else:
            vitals_labs2[col][row] = 1
                

In [ ]:
vitals_labs_icu_ID = vitals_labs_icu.ix[:,['ID_x', 'TIME_x']]
vitals_labs_icu_ID = pd.merge(vitals_labs_icu_ID,vitals_labs2, left_index=True,right_index=True, how='outer')
# vitals_labs_icu_ID.head(50)

In [ ]:
'''
Very slow approach
Fast approach is below

vitals_labs_icu_ID['LABEL'] = np.nan 
for dummy,id in list(enumerate(label_train.ID)):
    print id
    for row in vitals_labs_icu_ID.index:
        if vitals_labs_icu_ID.ID_x[row] == label_train.ID[id]:
           vitals_labs_icu_ID.LABEL = label_train.LABEL
    
vitals_labs_icu_ID.head(50)
'''

In [ ]:
vitals_labs_icu_ID['LABEL'] = np.nan 
for dummy,id in list(enumerate(label_train.ID)):
#     print id
#     print dummy
    id_index = vitals_labs_icu_ID.ID_x == id
    vitals_labs_icu_ID.LABEL[id_index] = label_train.LABEL[dummy]
    
print vitals_labs_icu_ID.head(10)

In [ ]:
# dummy
# label_train.LABEL[dummy]
# vitals_labs_icu_ID.tail(10)
# print vitals_labs_icu_ID.LABEL[628435]
# vitals_labs_icu_ID.LABEL[628435] = 1
# print vitals_labs_icu_ID.LABEL[628435]

In [ ]:
# index_endTime = index_zero[1:]
# vitals_labs_endTime = vitals_labs2.loc[index_endTime]
# vitals_labs_endTime = vitals_labs_endTime.append(vitals_labs2[-1:],ignore_index=True)

In [ ]:
# age_label_vitals_labs_endTime = pd.merge(age_train,vitals_labs_endTime, left_index=True,right_index=True, how='outer')
# age_label_vitals_labs_endTime.head(50)

In [ ]:
# create train feature
X_train = vitals_labs_icu_ID.iloc[:3001,2:33]
y_train = vitals_labs_icu_ID.iloc[:3001,34]
# assert X_train.shape[0] == y_train.size
# fit model
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
# Test data
X_test = vitals_labs_icu_ID.iloc[3001:,2:33]
# predict test data
predict_out = clf.predict(X_test)
# predict_proba.size

In [ ]:
temp_ID = vitals_labs_icu_ID.ix[:,['ID_x']]
temp_predict = pd.merge(temp_ID, predict_out, left_index=True,right_index=True, how='outer')

In [ ]:
# pos_idx = np.where(clf.classes_ == True)[0][0]
# test_df = pd.DataFrame(columns='predict')
# test_df = df_.fillna()
# test_df['predict'] = predict_proba[:,pos_idx]
test_match = (predict_proba == np.array(vitals_labs_icu_ID[3001:,34]))
percentage = float(np.sum(test_match)/np.size(predict_proba))*100
print percentage